### Joslin

* [Dataset]

### Setup Environment:

In [1]:
import os
import pandas as pd

from src.classifiers import preprocess_data, process_labels,split_data

from src.classifiers import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_cpu_metrics import train_early_fusion, train_late_fusion

In [2]:
PATH = 'Embeddings/joslin/'
COLUMN = 'embeddings'

In [3]:
text_path, images_path = 'text_embeddings.csv', 'Embeddings_dinov2_base.csv' # os.listdir(PATH)

## Get data

### Text

In [4]:
text = pd.read_csv(os.path.join(PATH, text_path))
text

,ID,EYE_DR,text,DR_2,DR_3,split,embeddings
0,67870,2,An image from the left eye of a female patien...,1,1,train,"[0.8949114084243774, 0.6178004741668701, -1.34..."
1,19446,2,An image from the left eye of a female patien...,1,1,train,"[1.0349855422973633, 1.2772152423858643, -1.71..."
2,113968,2,An image from the right eye of a male patient...,1,1,train,"[0.7820802927017212, 0.5888915657997131, -0.89..."
3,50528,1,An image from the right eye of a male patient...,1,1,train,"[0.9266375303268433, 1.2518030405044556, -1.40..."
4,91254,0,An image from the right eye of a male American...,0,0,train,"[1.0237936973571777, 1.0459136962890625, -0.75..."
...,...,...,...,...,...,...,...
24095,148958,1,"An image from the left eye of a male patient,...",1,1,train,"[0.9834482073783875, 1.3777140378952026, -1.40..."
24096,96198,0,An image from the right eye of a no sex report...,0,0,train,"[1.0272547006607056, 1.4537794589996338, 0.309..."
24097,82724,2,An image from the left eye of a female White p...,1,1,train,"[0.5230445861816406, 0.7840505838394165, -1.13..."
24098,173196,1,An image from the left eye of a female patien...,1,1,train,"[0.9105090498924255, 1.2843393087387085, -1.58..."


In [17]:
df = text
df

,ID,EYE_DR,text,DR_2,DR_3,split,embeddings
0,67870,2,An image from the left eye of a female patien...,1,1,train,"[0.8949114084243774, 0.6178004741668701, -1.34..."
1,19446,2,An image from the left eye of a female patien...,1,1,train,"[1.0349855422973633, 1.2772152423858643, -1.71..."
2,113968,2,An image from the right eye of a male patient...,1,1,train,"[0.7820802927017212, 0.5888915657997131, -0.89..."
3,50528,1,An image from the right eye of a male patient...,1,1,train,"[0.9266375303268433, 1.2518030405044556, -1.40..."
4,91254,0,An image from the right eye of a male American...,0,0,train,"[1.0237936973571777, 1.0459136962890625, -0.75..."
...,...,...,...,...,...,...,...
24095,148958,1,"An image from the left eye of a male patient,...",1,1,train,"[0.9834482073783875, 1.3777140378952026, -1.40..."
24096,96198,0,An image from the right eye of a no sex report...,0,0,train,"[1.0272547006607056, 1.4537794589996338, 0.309..."
24097,82724,2,An image from the left eye of a female White p...,1,1,train,"[0.5230445861816406, 0.7840505838394165, -1.13..."
24098,173196,1,An image from the left eye of a female patien...,1,1,train,"[0.9105090498924255, 1.2843393087387085, -1.58..."


### Images

In [6]:
images = pd.read_csv(os.path.join(PATH, images_path))
images.head()

,ImageName,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,122786.jpeg,-1.659120,2.434443,5.408528,-1.473753,-1.231571,-1.579182,0.310355,0.089211,-2.054411,...,-0.834506,-1.625948,-0.832199,0.383083,3.018108,-0.377753,3.330745,-0.877753,0.706453,-0.888934
1,108083.jpeg,0.649771,2.682362,3.609813,-0.099313,-3.025293,-1.844079,0.110555,1.285531,-3.551425,...,-1.258714,-1.734028,-0.291844,0.766295,3.141811,-0.881183,0.988453,-1.971897,0.635810,-2.960828
2,139103.jpeg,-0.452875,1.959501,5.043268,-1.375569,-2.404718,-0.326342,0.193852,-1.271304,-2.489283,...,-0.552394,-2.835432,0.370812,-0.378938,3.269454,-0.024993,2.338004,-1.397426,1.011528,-2.073067
3,104950.jpeg,0.243488,1.284947,3.641525,-1.265688,-1.907023,-1.053229,0.491267,-0.739931,-1.183857,...,-0.259700,-3.011966,-0.380113,0.168580,3.681773,0.684641,3.809679,-1.039821,1.166576,-1.322446
4,140898.jpeg,0.992673,1.722367,3.338633,-0.978513,-2.407126,-1.316537,1.006404,0.145190,-3.242617,...,-0.961301,-2.410475,0.398771,-1.165674,3.625564,-0.225109,2.984762,-1.122606,-0.683079,-2.567407


In [7]:
images.columns

Index(['ImageName', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '758', '759', '760', '761', '762', '763', '764', '765', '766', '767'],
      dtype='object', length=769)

In [8]:
text.columns

Index(['ID', 'EYE_DR', 'text', 'DR_2', 'DR_3', 'split', 'embeddings'], dtype='object')

### Merge and preprocess the datasets

In [ ]:
df = preprocess_data(text, images,text_id= "ID", image_id="ImageName")
df.drop(columns='text', inplace=True)
df.DR_3 = df.DR_3.astype(str)
df.to_csv(os.path.join(PATH, 'embeddings.csv'), index=False)
df.head()

S: check str


## Data Perparation

In [ ]:
# Split the data
train_df, test_df = split_data(df)

# Select features and labels vectors
text_columns = [column for column in df.columns if 'text' in column] #[f'text_{i}']
image_columns = [column for column in df.columns if 'image' in column] #[f'image_{i}']
label_columns = 'DR_3'


# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)


train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
df

In [ ]:
text_columns

### Models

In [ ]:
text_input_size = len(text_columns)
image_input_size = len(image_columns)
if label_columns == 'DR_2':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [ ]:
# Train early fusion model
print("Training Early Fusion Model:")
train_early_fusion(train_loader, test_loader, text_input_size, image_input_size, output_size, num_epochs=50, multilabel=multilabel, report=True)

In [ ]:
# Train late fusion model
print("Training Late Fusion Model:")
train_late_fusion(train_loader, test_loader, text_input_size, image_input_size, output_size, num_epochs=50, multilabel=multilabel, report=True)